In [40]:
import numpy as np
import sib_ldsc_z as ld
from scipy.optimize import minimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats
from importlib import reload
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import glob
reload(ld)

<module 'sib_ldsc_z' from 'c:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\inferz\\sib_ldsc_z.py'>

# Defining the PDF and the Log Likelihoods

The likelihood for a SNP $i$ is:

$$
l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) - \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i
$$

And its derivative:

$$
\frac{dl}{dV} = r_i S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}
$$

In [41]:
np.random.seed(123)

# N = int(100)
# S_size = int(N/2)
# S = np.array([np.array([[.5, 0], [0, .8]]),
#     np.array([[0.5, 0], [0, 0.8]])] * S_size )/N
# V = np.identity(2) * 0.5

N = int(1e4)
S = np.array([[[1e-4, -5 * 1e-5], [-5 * 1e-5, 1e-4]]] * N)
V = np.array([[0.5, 0.25], [0.25, 0.5]])


# N = int(1e4)
# S = np.array([[[1e-4, -5 * 1e-5, -4 * 1e-5], [-5 * 1e-5, 1e-4, -4 * 1e-5], [-5 * 1e-5, -4 * 1e-5, 1e-4]]] * N)
# V = np.array([[0.5, 0.25, 0.25], [0.25, 0.5, 0.25], [0.25, 0.25, 0.5]])

model = ld.sibreg(S = S)
model.simdata(V/N, N)

Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r


In [43]:
# Testing derivatives
aderiv = ld._grad_ll_v(ld.Vmat2V(V, N), model.z[0, :], model.S[0],  model.r[0], N)

nderiv = ld._num_grad_V(ld.Vmat2V(V, N), model.z[0, :], model.S[0], model.r[0],  N)

np.allclose(aderiv, nderiv)

True

In [44]:
# solving
output, result = model.solve()
print(result)

No initial guess provided.
Making 'optimal' matrix
Logll : 35239.41726772389
V : [1.e-06 1.e-06 0.e+00]
Logll : 32415.89683574153
V : [6.82502259e-01 7.30885418e-01 1.30583376e-06]
Logll : 32397.446580681
V : [0.66514028 0.72000816 0.03559895]
Logll : 32341.463045050004
V : [0.59569237 0.67649912 0.17798952]
Logll : 32300.142562068013
V : [0.46317375 0.55327355 0.39330531]
Logll : 32297.084300027993
V : [0.48258623 0.53512273 0.45274442]
Logll : 32296.55064823351
V : [0.46844001 0.53844819 0.46978596]
Logll : 32296.48299330651
V : [0.46348056 0.54285594 0.47613897]
Logll : 32296.482906011857
V : [0.46337656 0.54258746 0.47642431]
Logll : 32296.482905854835
V : [0.4633824  0.5425836  0.47643491]
Logll : 32296.48290585475
V : [0.4633823  0.54258363 0.47643523]
      fun: 32296.48290585475
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.46553826e-06, -2.97458568e-06, -1.89015940e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 11

In [47]:
output

{'v1': 0.46338230061859587, 'v2': 0.5425836300794872, 'r': 0.4764352261560131}